In [1]:
import numpy as np
import pandas as pd
import main

конвертация las в csv

In [ ]:
import GlobalConvertFunctions

keys_dict = {GlobalConvertFunctions.kid_well:"WELL_NAME",  
             GlobalConvertFunctions.kid_uwi:"WELL_NAME_UWI",
             GlobalConvertFunctions.kid_depth: "DEPT"}
params_name = ['APS', 'KGL']
i = 3
for params in params_name:
    keys_dict[i] = params
    i = i +1
print(keys_dict.keys())
print(keys_dict.values())

src_las_dir = "C:\\WORK\\ML\\tasks\\task 6\\test_data"
las_out_file_name = 'test_data\\las_test_out.csv'

print('Convert start')
GlobalConvertFunctions.create_csv_from_las(src_las_dir, las_out_file_name, keys_dict)
print('Convert end')

print("create_csv_from_las end")

In [2]:
las_data = pd.read_csv('test_data\\las_test_out.csv', delimiter=';')

del las_data['WELL_NAME_UWI']
del las_data['KGL']
las_data = las_data.rename(columns={'DEPT': 'DEPTH'})

print(len(np.unique(las_data['WELL_NAME'].tolist())))
las_data = las_data.dropna()
print(len(np.unique(las_data['WELL_NAME'].tolist())))

C:\Users\M1\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


19
16


In [3]:
las_data.replace(' ', '', regex=True, inplace=True)
las_data.replace('314_7po', '7PO', regex=True, inplace=True)
las_data.replace('215st2', '215ST2', regex=True, inplace=True)

Кровля и подошва

In [4]:
real_bottom = pd.read_csv('test_data\\bottom_df.csv', delimiter=';')
# real_bottom = real_bottom.rename(columns={'WELL_NAME': 'WELL_NAME_UWI'})
del real_bottom['X']
del real_bottom['Y']
real_bottom.head(n=2)

,WELL_NAME,DEPTH_BOTTOM
0,100,2564.81
1,101,2571.71


In [5]:
real_top = pd.read_csv('test_data\\top_df.csv', delimiter=';')
# real_top = real_top.rename(columns={'WELL_NAME': 'WELL_NAME_UWI'})
del real_top['X']
del real_top['Y']
real_top.head(n=2)

,WELL_NAME,DEPTH_TOP
0,100,2548.81
1,101,2556.23


In [6]:
real_top_bottom = pd.merge(real_top, real_bottom, on='WELL_NAME')

In [7]:
real_top_bottom.replace(' ', '', regex=True, inplace=True)
real_top_bottom.head()

,WELL_NAME,DEPTH_TOP,DEPTH_BOTTOM
0,100,2548.81,2564.81
1,101,2556.23,2571.71
2,102,2542.13,2555.93
3,103,2541.53,2558.09
4,103ST2,2566.04,2586.01


In [8]:
# real_top_bottom = real_top_bottom[ real_top_bottom['WELL_NAME'] in np.unique(las_data['WELL_NAME'].tolist())]
real_top_bottom['WELL_NAME'] = real_top_bottom['WELL_NAME'].astype('str')
las_data['WELL_NAME'] = las_data['WELL_NAME'].astype('str')

In [9]:
np.unique(las_data['WELL_NAME'].tolist())

array(['135', '137', '149', '166', '171', '187', '207', '215ST2', '232',
       '244', '338', '383', '451', '602', '604', '7PO'], 
      dtype='<U6')

In [10]:
real_data = pd.merge(las_data, real_top_bottom, on='WELL_NAME')
real_data.head()

,WELL_NAME,DEPTH,APS,DEPTH_TOP,DEPTH_BOTTOM
0,135,2776.0,0.0530,2549.5,2572.27
1,135,2776.1,0.0655,2549.5,2572.27
2,135,2776.2,0.0780,2549.5,2572.27
3,135,2776.3,0.0840,2549.5,2572.27
4,135,2776.4,0.0900,2549.5,2572.27


In [11]:
np.unique(real_data['WELL_NAME'].tolist())

array(['135', '137', '149', '166', '171', '187', '207', '215ST2', '232',
       '244', '338', '383', '451', '602', '604', '7PO'], 
      dtype='<U6')

Загружаем траектории

In [12]:
trace = pd.read_csv(main.data_dir + "\\trace.csv", delimiter=';')
trace.head()

,UWI,well_name,X,Y,MD,Z
0,314_100,100,354 727.390,6 470 017.860,10.000,-110.780
1,314_100,100,354 727.400,6 470 017.870,20.000,-100.780
2,314_100,100,354 727.410,6 470 017.880,30.000,-90.780
3,314_100,100,354 727.430,6 470 017.900,40.000,-80.780
4,314_100,100,354 727.460,6 470 017.930,50.000,-70.780


In [13]:
del trace['UWI']
trace.replace(' ', '', regex=True, inplace=True)
trace['MD'] = trace['MD'].astype('float64')
trace['X'] = trace['X'].astype('float64')
trace['Y'] = trace['Y'].astype('float64')
trace['Z'] = trace['Z'].astype('float64')
trace.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69563 entries, 0 to 69562
Data columns (total 5 columns):
well_name    69563 non-null object
X            69563 non-null float64
Y            69563 non-null float64
MD           69545 non-null float64
Z            69563 non-null float64
dtypes: float64(4), object(1)
memory usage: 2.7+ MB


In [14]:
wells_list = np.unique(real_data['WELL_NAME'].tolist())
dev_path = "C:\\WORK\\ML\\las_ml\\test_data\\dev\\"
ext = "dev"
for well in wells_list:    
    well_trace = trace[trace['well_name'] == well]
    main.convert_trace_to_dev(well_trace, dev_path, ext)

In [15]:
import lasio
import re
def load_and_convert_to_interp(dev_path, well_name):
    f = open(dev_path + well_name + '.dev', 'r')
    well_num = 0
    md = []
    abs = []
    for line in f.readlines():
        if well_num > 16:
            # list = line.split(' ')
            # print(list)
            numbers = re.findall(r'[-]?[0-9]+.[0-9]+', line)
            md.append(float(numbers[0]))
            abs.append(float(numbers[3]))
        well_num = well_num +1
    f.close()
    f_spline = interpolate.interp1d(abs, md, kind = 'slinear', bounds_error=False)
    return f_spline

In [20]:
from scipy import interpolate
well_name_list = wells_list
well_values = []
x_values = []
# y_values = []
count_val = 100
well_count = 0
min_count_val = 10
for well_name in well_name_list:
    if well_count%20 == 0:
        print(well_count, ' of ', len(well_name_list))
    print(well_name)
    data_well =  real_data[real_data['WELL_NAME'] == well_name]
    f_spline = load_and_convert_to_interp(dev_path, well_name)
    bottom = f_spline(data_well['DEPTH_BOTTOM'].tolist()[0])
    top = f_spline(data_well['DEPTH_TOP'].tolist()[0])
    data_well_by_bound = data_well[(data_well['DEPTH'] >= top) & (data_well['DEPTH'] <= bottom)]
    x_arr = data_well_by_bound['DEPTH']
    y_arr = data_well_by_bound['APS']
    
    # print ('length of array depth', len(x_arr))
    # print(len(x_arr), len(y_arr))
    if len(x_arr) < min_count_val:
        print(well_name + 'мало данных')
        continue
    f_spline = interpolate.interp1d(x_arr, y_arr, kind = 'slinear')
    h_start = data_well_by_bound['DEPTH'].min()
    h_end = data_well_by_bound['DEPTH'].max()
    # print(h_start, h_end, top, bottom)
    h_step = (h_end - h_start)/count_val
    x_temp = []
    i = 0
    while(i < count_val):
        x_temp.append(float(f_spline(h_start + i * h_step)))
        i = i + 1
    x_values.append(x_temp)  
    well_values.append(well_name)
#     y_values.append(data_well['WC'].tolist()[0])
    well_count = well_count + 1
print('end fill')

0  of  16
135
137
149
166
171
187
207
207мало данных
215ST2
232
244
338
383
451
602
604
7PO
end fill


In [21]:
np.shape(x_values)

(15, 100)

Решение с использованием ансамбля

In [22]:
import pickle
# Loading the saved decision tree model pickle
decision_tree_pkl_filename = 'test_data\\decision_forest_grid_fake_data_regressor.pkl'
decision_tree_model_pkl_load = open(decision_tree_pkl_filename, 'rb')
decision_tree_model = pickle.load(decision_tree_model_pkl_load)
print("Loaded Decision tree model :: ", decision_tree_model)

Loaded Decision tree model ::  GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
           oob_score=False, random_state=17, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_depth': [1, 10, 20, 40, 60], 'n_estimators': [1, 10, 30, 50, 100, 200, 500], 'max_features': ['auto', 'sqrt', 'log2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=True)


C:\Users\M1\Anaconda3\lib\site-packages\sklearn\base.py:312: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.18.1 when using version 0.19.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\M1\Anaconda3\lib\site-packages\sklearn\base.py:312: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 0.18.1 when using version 0.19.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\M1\Anaconda3\lib\site-packages\sklearn\base.py:312: UserWarning: Trying to unpickle estimator GridSearchCV from version 0.18.1 when using version 0.19.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [23]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.cross_validation import cross_val_score
y_pred_tree = decision_tree_model.predict(x_values)

In [24]:
y_pred_tree

array([ 0.33966,  0.3547 ,  0.28292,  0.28328,  0.3321 ,  0.33658,
        0.33438,  0.2969 ,  0.2932 ,  0.30154,  0.2889 ,  0.25072,
        0.26742,  0.329  ,  0.32314])

In [25]:
predict_tree = pd.DataFrame({'wc_tree':y_pred_tree, 'well':well_values})
predict_tree.head()

,wc_tree,well
0,0.33966,135
1,0.35470,137
2,0.28292,149
3,0.28328,166
4,0.33210,171


Решение с использованием нейросети

In [26]:
from keras.datasets import boston_housing
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.models import model_from_json
# Устанавливаем seed для повторяемости результатов
np.random.seed(42)
# Загружаем данные об архитектуре сети из файла json
json_file = open("fake_aps_model.json", "r")
loaded_model_json = json_file.read()
json_file.close()
# Создаем модель на основе загруженных данных
loaded_model = model_from_json(loaded_model_json)
# Загружаем веса в модель
loaded_model.load_weights("fake_aps_model.h5")
# Компилируем модель
loaded_model.compile(loss="mse", optimizer="adam", metrics=['mae'])
print(loaded_model.summary())

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 250)               25250     
_________________________________________________________________
dense_2 (Dense)              (None, 200)               50200     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               20100     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 49)                4949      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 50        
Total para

In [27]:
y_pred_neural = loaded_model.predict(x_values)
y_pred = []
for x in y_pred_neural:
    y_pred.append(x[0])
y_pred
predict_neural = pd.DataFrame({'wc_neural':y_pred, 'well':well_values})
predict_neural.head()

,wc_neural,well
0,0.353897,135
1,0.355354,137
2,0.264317,149
3,0.206458,166
4,0.327339,171


In [28]:
predict = pd.merge(predict_neural, predict_tree, on='well')

In [29]:
predict.to_csv('test_data/predict.csv', index=False, sep = ';')